# Toronto Neighborhood Clustering Analysis

In [221]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [222]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})
ths = table.find_all('th')

In [223]:
headings = [th.text.strip() for th in ths]
print(headings)

['Postcode', 'Borough', 'Neighbourhood']


In [224]:
Table_Data = []
for tr in table.find_all('tr'):
    tds = tr.find_all('td')
    if not tds:
        continue
    Postcode, Borough, Neighborhood = [td.text.strip() for td in tds[:3]]
    Table_Data.append([Postcode, Borough, Neighborhood])
    
Table_Data_Grouped = []
for row in Table_Data:
    entry = []

df = pd.DataFrame(Table_Data)
df.columns = headings 

df = df[df.Borough != 'Not assigned'] # This removes all entries which have no Borough name

df.loc[df['Neighbourhood'] == 'Not assigned','Neighbourhood'] = df['Borough'] # This replaces Neighbourhood name with Borough name if Neighbourhood name is not assigned

neib = lambda a: ", ".join(a)
df = df.groupby(by='Postcode').agg({'Borough': 'max','Neighbourhood': neib}).reset_index() # This groups neighbourhoods with the same postcode

df.tail()

,Postcode,Borough,Neighbourhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
102,M9W,Etobicoke,Northwest


In [225]:
df.shape

(103, 3)

In [226]:
url = 'https://cocl.us/Geospatial_data'
geo=pd.read_csv(url)
geo.tail()

,Postal Code,Latitude,Longitude
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


In [227]:
dfl = pd.concat([df, geo['Latitude'], geo['Longitude']], axis=1, sort=False)
dfl.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
